In [ ]:
import torch
from torch import nn
from torch.optim import lr_scheduler
import pandas as pd
import torch.nn.functional as F
import torchvision.transforms as transforms  
import torchvision
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import cv2
import pandas as pd
import torchvision.transforms as transforms 
from torchvision.transforms import ToTensor,Normalize, RandomHorizontalFlip, Resize
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torch.autograd import Variable
from torchvision import datasets, models, transforms
import torch.optim as optim
import time
import torch.backends.cudnn as cudnn
import copy
cudnn.benchmark = True
plt.ion()

In [ ]:
pred_dir='../input/sprints-ai-and-ml-competition-2022/Scenes testing test/Scenes testing test'
train_dir ='../input/sprints-ai-and-ml-competition-2022/Scenes training set/Scenes training set'

pred = [os.path.join(pred_dir, f) for f in os.listdir(pred_dir)]

In [ ]:
pip install split-folders

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import splitfolders
splitfolders.ratio(train_dir, output="output",ratio=(.85, 0.15))

Copying files: 14034 files [00:13, 1017.55 files/s]


In [ ]:
data_transforms = {
    'train': torchvision.transforms.Compose([transforms.Resize((150,150)),
                                      transforms.ColorJitter(),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),
                                      transforms.
                                      transforms.Normalize([0.5, 0.5, 0.5],
                                                           [0.5, 0.5, 0.5])
                                      ])
,
    'val':torchvision.transforms.Compose([transforms.Resize((150,150)),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.5, 0.5, 0.5],
                                                           [0.5, 0.5, 0.5])])}

data_dir = './output'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [ ]:
model_conv = torchvision.models.convnext_small(pretrained=True)
model_conv.classifier[2] = nn.Linear(model_conv.classifier[2].in_features, 6)
model_conv = model_conv.to(device)

In [ ]:

criterion = nn.CrossEntropyLoss()
optimizer_conv =torch.optim.SGD(model_conv.parameters(), lr=0.001)
exp_lr_scheduler =  torch.optim.lr_scheduler.MultiStepLR(optimizer_conv, milestones=[4,6], gamma=0.06)

In [ ]:
model_conv = train_model(model_conv, criterion, optimizer_conv,
                         exp_lr_scheduler, num_epochs=8)

Epoch 0/7
----------
train Loss: 0.3304 Acc: 0.8888
val Loss: 0.2244 Acc: 0.9255

Epoch 1/7
----------
train Loss: 0.2061 Acc: 0.9276
val Loss: 0.1829 Acc: 0.9402

Epoch 2/7
----------
train Loss: 0.1753 Acc: 0.9391
val Loss: 0.1723 Acc: 0.9407

Epoch 3/7
----------
train Loss: 0.1620 Acc: 0.9432
val Loss: 0.1746 Acc: 0.9393

Epoch 4/7
----------
train Loss: 0.1301 Acc: 0.9551
val Loss: 0.1633 Acc: 0.9435

Epoch 5/7
----------
train Loss: 0.1232 Acc: 0.9568
val Loss: 0.1633 Acc: 0.9435

Epoch 6/7
----------
train Loss: 0.1194 Acc: 0.9588
val Loss: 0.1624 Acc: 0.9440

Epoch 7/7
----------
train Loss: 0.1213 Acc: 0.9579
val Loss: 0.1620 Acc: 0.9440

Training complete in 26m 57s
Best val Acc: 0.944023


In [ ]:
def prediction(im):
    img_tens = data_transforms['val'](im)
    img_im = img_tens.unsqueeze(0).cuda() 
    uinput = Variable(img_im)
    uinput = uinput.to(device)
    out = model_conv(uinput)
    out = out.data.cpu().numpy().argmax()    
    return out

In [ ]:
lis=[]
for x in pred:
    img = Image.open(x)
    out = prediction(img)
    lis.append(out)
df = {'Image':os.listdir(pred_dir),'Label':lis}
df=pd.DataFrame(df)

In [ ]:
df.to_csv('submit.csv',index=False)